# basic function

In [10]:
!pip install python-dotenv google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 14.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [google-genai] [google-genai]


In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
assert api_key is not None

In [13]:
from google import genai
client = genai.Client()

In [14]:
SAMPLE_TEXT = """
The impact of artificial intelligence on scientific research has been profound. According to a study 
published in Nature (Smith et al., 2023, "AI-Driven Discovery in Materials Science", 
DOI: 10.1038/s41586-023-06221-2), machine learning models have accelerated materials discovery by 50x.

This trend is also reflected in social media discussions. A viral Twitter thread by @AIResearcher 
(https://twitter.com/AIResearcher/status/1234567890) garnered over 50,000 likes discussing how 
GPT-4 is being used in drug discovery pipelines.

The New York Times reported on this phenomenon in their article "The AI Revolution in Labs" 
(https://www.nytimes.com/2023/08/15/science/ai-laboratory-research.html), noting that major 
pharmaceutical companies are investing billions.

A Reddit discussion on r/MachineLearning (https://reddit.com/r/MachineLearning/comments/abc123) 
with 2,400 upvotes debated the reproducibility concerns raised in the original Nature paper.

For a technical deep-dive, the arXiv preprint "Transformer Architectures for Scientific Computing" 
(arXiv:2401.12345) provides implementation details. The official documentation at 
https://docs.example-ai-tool.com/guide has step-by-step tutorials.

Personal blog post by Dr. Jane Chen (https://janechenml.substack.com/p/ai-in-science-2024) 
offers a practitioner's perspective with 15,000 views.

The CDC guidelines (https://www.cdc.gov/ai-health/guidelines.html) now include recommendations 
for AI-assisted diagnostics.
"""

In [15]:
EXTRACTION_PROMPT = """
You are a reference extraction system. Analyze the following text and extract ALL references, citations, and sources mentioned.

For each reference found, provide:
1. raw_text: The exact text/citation as it appears
2. type: One of: academic_journal, academic_preprint, news_media, government_official, social_media, personal_blog, documentation, unknown
3. url: The URL if present (null if not)
4. doi: The DOI if present (null if not)
5. platform: The platform/source (e.g., "Nature", "Twitter", "arXiv", "Reddit", "NYTimes", etc.)
6. engagement_signals: Any mentioned metrics like views, likes, upvotes (null if not mentioned)

Return the results as a JSON array. Be thorough - extract every reference, even implicit ones.

TEXT TO ANALYZE:
---
{text}
---

Return ONLY valid JSON, no markdown formatting or explanation.
"""

In [16]:
import json

def extract_references(text: str) -> list[dict]:
    """Extract references from text using Gemini."""
    
    prompt = EXTRACTION_PROMPT.format(text=text)
    
    response = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=prompt
    )
    
    # Parse the JSON response
    try:
        response_text = response.text.strip()
        # Clean up markdown code blocks if present
        if response_text.startswith("```json"):
            response_text = response_text[7:]
        if response_text.startswith("```"):
            response_text = response_text[3:]
        if response_text.endswith("```"):
            response_text = response_text[:-3]
        
        return json.loads(response_text.strip())
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        print(f"Raw response: {response.text}")
        return []

In [17]:
references = extract_references(SAMPLE_TEXT)
print(f"Found {len(references)} references:\n")
print(json.dumps(references, indent=2))

Found 8 references:

[
  {
    "raw_text": "Nature (Smith et al., 2023, \"AI-Driven Discovery in Materials Science\", DOI: 10.1038/s41586-023-06221-2)",
    "type": "academic_journal",
    "url": null,
    "doi": "10.1038/s41586-023-06221-2",
    "platform": "Nature",
    "engagement_signals": null
  },
  {
    "raw_text": "@AIResearcher (https://twitter.com/AIResearcher/status/1234567890)",
    "type": "social_media",
    "url": "https://twitter.com/AIResearcher/status/1234567890",
    "doi": null,
    "platform": "Twitter",
    "engagement_signals": "50,000 likes"
  },
  {
    "raw_text": "The New York Times reported on this phenomenon in their article \"The AI Revolution in Labs\" (https://www.nytimes.com/2023/08/15/science/ai-laboratory-research.html)",
    "type": "news_media",
    "url": "https://www.nytimes.com/2023/08/15/science/ai-laboratory-research.html",
    "doi": null,
    "platform": "NYTimes",
    "engagement_signals": null
  },
  {
    "raw_text": "Reddit discussion on

reflections:
- data structure encourages hallucination - have not defined web search as a tool, so engagement_signals can be hallucinated.
- may want to start with a digital heuristic - EEAT, experience expertise authorativeness trustworthiness
- will want to log tokens in, tokens out, latency, to enable evaluation.
- may want to start an examples folder with real outputs we've evaluated (to enable quality ratings) - or find another example elsewhere.

# Iteration 2
Get logging, improve prompt, provide outputs for UX elements

In [25]:
import os
import json
import time
from dataclasses import dataclass, asdict
from datetime import datetime
from dotenv import load_dotenv
from google import genai

load_dotenv()

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

@dataclass
class LLMLog:
    timestamp: str
    model: str
    prompt_tokens: int
    completion_tokens: int
    total_tokens: int
    latency_ms: int
    
    def to_dict(self):
        return asdict(self)

# Store logs for this session
llm_logs: list[LLMLog] = []

def generate_with_logging(prompt: str, model: str = "gemini-2.0-flash-exp") -> tuple[str, LLMLog]:
    """Generate content and log token usage + latency."""
    
    start_time = time.time()
    
    response = client.models.generate_content(
        model=model,
        contents=prompt
    )
    
    latency_ms = int((time.time() - start_time) * 1000)
    
    usage = response.usage_metadata
    log = LLMLog(
        timestamp=datetime.now().isoformat(),
        model=model,
        prompt_tokens=usage.prompt_token_count,
        completion_tokens=usage.candidates_token_count,
        total_tokens=usage.total_token_count,
        latency_ms=latency_ms
    )
    
    llm_logs.append(log)
    
    return response.text, log

print("✅ generate_with_logging defined")

print("✅ Setup complete")

✅ generate_with_logging defined
✅ Setup complete


In [20]:

with open('example_question.txt',mode='r') as file:
    EXAMPLE_QUESTION = file.read()

with open('example_answer.txt',mode='r') as file:
    EXAMPLE_ANSWER = file.read()

In [21]:
from dataclasses import dataclass, field
from typing import Optional
from enum import Enum

class SourceType(str, Enum):
    ACADEMIC_JOURNAL = "academic_journal"
    ACADEMIC_PREPRINT = "academic_preprint"
    NEWS_MEDIA = "news_media"
    GOVERNMENT = "government"
    SOCIAL_MEDIA = "social_media"
    PERSONAL_BLOG = "personal_blog"
    DOCUMENTATION = "documentation"
    COMMERCIAL = "commercial"
    UNKNOWN = "unknown"

@dataclass
class ExtractedReference:
    """Stage 1 output - extracted from text only, no inference."""
    raw_text: str
    url: Optional[str] = None
    doi: Optional[str] = None
    arxiv_id: Optional[str] = None
    source_type: SourceType = SourceType.UNKNOWN
    platform_name: Optional[str] = None
    # Only what's explicitly in text
    stated_author: Optional[str] = None
    stated_date: Optional[str] = None
    stated_metrics: Optional[str] = None  # e.g., "2,400 upvotes" as string

@dataclass 
class CRAAPScore:
    """Stage 2 output - requires verification."""
    score: int  # 1-5
    evidence: str  # Why this score
    
@dataclass
class VerifiedReference:
    """Stage 2 output - after search/verification."""
    reference: ExtractedReference
    url_accessible: Optional[bool] = None
    currency: Optional[CRAAPScore] = None
    relevance: Optional[CRAAPScore] = None
    authority: Optional[CRAAPScore] = None
    accuracy: Optional[CRAAPScore] = None
    purpose: Optional[CRAAPScore] = None
    overall_score: Optional[float] = None
    verification_notes: str = ""

@dataclass
class AnalysisInput:
    """Full context for analysis."""
    question: str
    answer: str
    context: dict = field(default_factory=dict)  # e.g., {"user_location": "AU", "domain": "healthcare"}

print("✅ Data structures defined")

✅ Data structures defined


In [22]:
EXTRACTION_PROMPT_V2 = """
You are a reference extraction system. Extract ALL references from an AI-generated answer.

CONTEXT:
- User's question: {question}
- User's context: {context}

INSTRUCTIONS:
Extract every reference, citation, URL, or source mentioned in the answer.
Only extract information EXPLICITLY stated. Use null for anything not directly written.

For each reference provide:
- raw_text: Exact citation as written
- url: URL if present, null otherwise
- doi: DOI if present, null otherwise  
- arxiv_id: arXiv ID if present, null otherwise
- source_type: One of: academic_journal, academic_preprint, news_media, government, social_media, personal_blog, documentation, commercial, unknown
- platform_name: Specific platform/publication name
- stated_author: Author name if explicitly written, null otherwise
- stated_date: Publication date/year if written, null otherwise
- stated_metrics: Any engagement metrics as written (e.g., "50k views"), null otherwise

ANSWER TO ANALYZE:
---
{answer}
---

Return ONLY a JSON array. Do not hallucinate - null for anything not explicit.
"""

def extract_references_v2(input_data: AnalysisInput) -> tuple[list[dict], LLMLog]:
    """Extract references with question context."""
    
    prompt = EXTRACTION_PROMPT_V2.format(
        question=input_data.question,
        context=json.dumps(input_data.context),
        answer=input_data.answer
    )
    
    response_text, log = generate_with_logging(prompt)
    
    try:
        clean_text = response_text.strip()
        if clean_text.startswith("```json"):
            clean_text = clean_text[7:]
        if clean_text.startswith("```"):
            clean_text = clean_text[3:]
        if clean_text.endswith("```"):
            clean_text = clean_text[:-3]
        
        return json.loads(clean_text.strip()), log
    except json.JSONDecodeError as e:
        print(f"Error: {e}\nRaw: {response_text}")
        return [], log

print("✅ extract_references_v2 defined")

✅ extract_references_v2 defined


In [23]:
CRAAP_VERIFICATION_PROMPT = """
You are a source verification system evaluating references for a specific user question.

USER QUESTION: {question}
USER CONTEXT: {context}

REFERENCE TO VERIFY:
{reference_json}

Using web search, evaluate this reference on the CRAAP framework. For each dimension, provide:
- score: 1-5 (1=poor, 5=excellent)
- evidence: Specific reasons for the score

EVALUATION CRITERIA:

**Currency (1-5):**
- When was this published/updated?
- Is the information current enough for the topic?
- Are there more recent sources available?

**Relevance (1-5):**
- Does this source address the user's actual question?
- Is it geographically relevant? (User is in {user_location})
- Is it the right depth/audience level?

**Authority (1-5):**
- Who is the author? What are their credentials?
- Is the publisher/platform reputable for this topic?
- Is there institutional backing?

**Accuracy (1-5):**
- Can the claims be verified elsewhere?
- Does the source cite its own references?
- Is it peer-reviewed or edited?

**Purpose (1-5):**
- Why does this source exist? (inform/persuade/sell/entertain)
- Is there obvious bias or conflict of interest?
- Is it trying to sell something?

Also check:
- url_accessible: Can the URL be reached? (true/false/null if no URL)

Return JSON:
{{
  "url_accessible": true/false/null,
  "currency": {{"score": 1-5, "evidence": "..."}},
  "relevance": {{"score": 1-5, "evidence": "..."}},
  "authority": {{"score": 1-5, "evidence": "..."}},
  "accuracy": {{"score": 1-5, "evidence": "..."}},
  "purpose": {{"score": 1-5, "evidence": "..."}},
  "overall_score": 1.0-5.0,
  "verification_notes": "Key findings from search"
}}

Return ONLY valid JSON.
"""

def verify_reference_with_search(
    reference: dict, 
    question: str, 
    context: dict,
    model: str = "gemini-2.0-flash-exp"
) -> tuple[dict, LLMLog]:
    """Verify a single reference using Gemini with Google Search."""
    
    prompt = CRAAP_VERIFICATION_PROMPT.format(
        question=question,
        context=json.dumps(context),
        reference_json=json.dumps(reference, indent=2),
        user_location=context.get("user_location", "unknown")
    )
    
    # Enable Google Search grounding
    from google.genai import types
    
    start_time = time.time()
    
    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=types.GenerateContentConfig(
            tools=[types.Tool(google_search=types.GoogleSearch())]
        )
    )
    
    latency_ms = int((time.time() - start_time) * 1000)
    
    usage = response.usage_metadata
    log = LLMLog(
        timestamp=datetime.now().isoformat(),
        model=model,
        prompt_tokens=usage.prompt_token_count,
        completion_tokens=usage.candidates_token_count,
        total_tokens=usage.total_token_count,
        latency_ms=latency_ms
    )
    llm_logs.append(log)
    
    try:
        clean_text = response.text.strip()
        if clean_text.startswith("```json"):
            clean_text = clean_text[7:]
        if clean_text.startswith("```"):
            clean_text = clean_text[3:]
        if clean_text.endswith("```"):
            clean_text = clean_text[:-3]
        
        return json.loads(clean_text.strip()), log
    except json.JSONDecodeError as e:
        print(f"Error: {e}\nRaw: {response.text}")
        return {}, log

print("✅ verify_reference_with_search defined")

✅ verify_reference_with_search defined


In [26]:
# Replace with your actual example
test_input = AnalysisInput(
    question=EXAMPLE_QUESTION,
    answer=EXAMPLE_ANSWER,
    context={
        "user_location": "AU",
        "domain": "AI consulting",  # e.g., "healthcare", "finance", "technology"
    }
)

# Stage 1: Extract
print("=== STAGE 1: EXTRACTION ===")
refs, extract_log = extract_references_v2(test_input)
print(f"Tokens: {extract_log.total_tokens} | Latency: {extract_log.latency_ms}ms")
print(f"Found {len(refs)} references")
print(json.dumps(refs, indent=2))

=== STAGE 1: EXTRACTION ===
Tokens: 5678 | Latency: 10661ms
Found 16 references
[
  {
    "raw_text": "iTnews",
    "url": "https://www.itnews.com.au/feature/how-australian-businesses-can-overcome-the-struggle-to-move-ai-from-hype-to-reality-618510",
    "doi": null,
    "arxiv_id": null,
    "source_type": "news_media",
    "platform_name": "iTnews",
    "stated_author": null,
    "stated_date": null,
    "stated_metrics": null
  },
  {
    "raw_text": "Substack",
    "url": "https://futureofprospecting.substack.com/p/the-ai-threat-that-could-break-salesforce",
    "doi": null,
    "arxiv_id": null,
    "source_type": "personal_blog",
    "platform_name": "Substack",
    "stated_author": null,
    "stated_date": null,
    "stated_metrics": null
  },
  {
    "raw_text": "The Conversation",
    "url": "https://theconversation.com/australian-businesses-have-actually-been-slow-to-adopt-ai-survey-finds-269812",
    "doi": null,
    "arxiv_id": null,
    "source_type": "news_media",
    "pl

In [28]:
print("=== STAGE 2: VERIFICATION ===")

verified_refs = []
for i, ref in enumerate(refs[:3]):  # Limit to first 3 for testing
    print(f"\nVerifying {i+1}/{min(3, len(refs))}: {ref.get('platform_name', 'unknown')}")
    
    verification, verify_log = verify_reference_with_search(
        ref, 
        test_input.question,
        test_input.context
    )
    
    print(f"  Tokens: {verify_log.total_tokens} | Latency: {verify_log.latency_ms}ms")
    
    if verification:
        overall = verification.get('overall_score', 'N/A')
        print(f"  Overall CRAAP Score: {overall}/5")
        print(f"  Notes: {verification.get('verification_notes', '')[:100]}...")
    
    verified_refs.append({
        "reference": ref,
        "verification": verification
    })

print("\n=== FULL RESULTS ===")
print(json.dumps(verified_refs, indent=2))

=== STAGE 2: VERIFICATION ===

Verifying 1/3: iTnews
  Tokens: 1109 | Latency: 4178ms
  Overall CRAAP Score: 3.6/5
  Notes: The iTnews article seems generally relevant and authoritative, but the lack of a specific publicatio...

Verifying 2/3: Substack
  Tokens: 1124 | Latency: 4113ms
  Overall CRAAP Score: 2.2/5
  Notes: This Substack article discussing the AI threat to Salesforce is of limited value for researching the...

Verifying 3/3: The Conversation
  Tokens: 1018 | Latency: 3556ms
  Overall CRAAP Score: 4.0/5
  Notes: Key findings from search indicate that Australian businesses have been slow to adopt AI. This articl...

=== FULL RESULTS ===
[
  {
    "reference": {
      "raw_text": "iTnews",
      "url": "https://www.itnews.com.au/feature/how-australian-businesses-can-overcome-the-struggle-to-move-ai-from-hype-to-reality-618510",
      "doi": null,
      "arxiv_id": null,
      "source_type": "news_media",
      "platform_name": "iTnews",
      "stated_author": null,
      "s

quick check of logs

In [29]:
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 49.1 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 57.1 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]2m2/3 [pandas]


In [30]:
import pandas as pd
df_logs = pd.DataFrame([log.to_dict() for log in llm_logs])
df_logs

,timestamp,model,prompt_tokens,completion_tokens,total_tokens,latency_ms
0,2026-01-08T01:04:51.496310,gemini-2.0-flash-exp,3746,1932,5678,10661
1,2026-01-08T01:05:39.939271,gemini-2.0-flash-exp,651,458,1109,4178
2,2026-01-08T01:05:44.053101,gemini-2.0-flash-exp,627,497,1124,4113
3,2026-01-08T01:05:47.609839,gemini-2.0-flash-exp,636,382,1018,3556


# iteration 3
Include more logging, aggregate

In [1]:
import os
import json
import time
from dataclasses import dataclass, asdict
from datetime import datetime
from dotenv import load_dotenv
from google import genai

load_dotenv()

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

@dataclass
class LLMLog:
    timestamp: str
    model: str
    task: str
    prompt_tokens: int
    completion_tokens: int
    total_tokens: int
    latency_ms: int
    
    def to_dict(self):
        return asdict(self)

# Store logs for this session
llm_logs: list[LLMLog] = []

def generate_with_logging(prompt: str, task: str, model: str = "gemini-2.0-flash-exp") -> tuple[str, LLMLog]:
    """Generate content and log token usage + latency."""
    
    start_time = time.time()
    
    response = client.models.generate_content(
        model=model,
        contents=prompt
    )
    
    latency_ms = int((time.time() - start_time) * 1000)
    
    usage = response.usage_metadata
    log = LLMLog(
        timestamp=datetime.now().isoformat(),
        model=model,
        task=task,
        prompt_tokens=usage.prompt_token_count,
        completion_tokens=usage.candidates_token_count,
        total_tokens=usage.total_token_count,
        latency_ms=latency_ms
    )
    
    llm_logs.append(log)
    
    return response.text, log

print("✅ generate_with_logging defined")

print("✅ Setup complete")

✅ generate_with_logging defined
✅ Setup complete


In [2]:
with open('example_answer.txt',mode='r') as file:
    EXAMPLE_ANSWER = file.read()

In [3]:
from dataclasses import dataclass, field
from typing import Optional
from enum import Enum

class SourceType(str, Enum):
    ACADEMIC_JOURNAL = "academic_journal"
    ACADEMIC_PREPRINT = "academic_preprint"
    NEWS_MEDIA = "news_media"
    GOVERNMENT = "government"
    SOCIAL_MEDIA = "social_media"
    PERSONAL_BLOG = "personal_blog"
    DOCUMENTATION = "documentation"
    COMMERCIAL = "commercial"
    UNKNOWN = "unknown"

@dataclass
class ExtractedReference:
    """Stage 1 output - extracted from text only, no inference."""
    raw_text: str
    url: Optional[str] = None
    doi: Optional[str] = None
    arxiv_id: Optional[str] = None
    source_type: SourceType = SourceType.UNKNOWN
    platform_name: Optional[str] = None
    # Only what's explicitly in text
    stated_author: Optional[str] = None
    stated_date: Optional[str] = None
    stated_metrics: Optional[str] = None  # e.g., "2,400 upvotes" as string

@dataclass 
class CRAAPScore:
    """Stage 2 output - requires verification."""
    score: int  # 1-5
    evidence: str  # Why this score
    
@dataclass
class VerifiedReference:
    """Stage 2 output - after search/verification."""
    reference: ExtractedReference
    url_accessible: Optional[bool] = None
    currency: Optional[CRAAPScore] = None
    relevance: Optional[CRAAPScore] = None
    authority: Optional[CRAAPScore] = None
    accuracy: Optional[CRAAPScore] = None
    purpose: Optional[CRAAPScore] = None
    overall_score: Optional[float] = None
    verification_notes: str = ""


print("✅ Data structures defined")

✅ Data structures defined


In [6]:
EXTRACTION_PROMPT_V3 = """
You are a reference extraction system. Extract ALL references from this AI-generated text.

Only extract information EXPLICITLY stated. Use null for anything not directly written.

For each reference provide:
- raw_text: Exact citation as written
- url: URL if present, null otherwise
- doi: DOI if present, null otherwise  
- arxiv_id: arXiv ID if present, null otherwise
- source_type: One of: academic_journal, academic_preprint, news_media, government, social_media, personal_blog, documentation, commercial, unknown
- platform_name: Specific platform/publication name
- stated_author: Author name if explicitly written, null otherwise
- stated_date: Publication date/year if written, null otherwise
- stated_metrics: Any engagement metrics as written (e.g., "50k views"), null otherwise

TEXT TO ANALYZE:
---
{answer}
---

Return ONLY a JSON array. Do not hallucinate - null for anything not explicit.
"""

def extract_references(answer: str) -> tuple[list[dict], LLMLog]:
    """Extract references from answer text."""
    
    prompt = EXTRACTION_PROMPT_V3.format(answer=answer)
    response_text, log = generate_with_logging(prompt,task='extract_references')
    
    try:
        clean_text = response_text.strip()
        if clean_text.startswith("```json"):
            clean_text = clean_text[7:]
        if clean_text.startswith("```"):
            clean_text = clean_text[3:]
        if clean_text.endswith("```"):
            clean_text = clean_text[:-3]
        
        return json.loads(clean_text.strip()), log
    except json.JSONDecodeError as e:
        print(f"Error: {e}\nRaw: {response_text}")
        return [], log

print("✅ extract_references defined")

✅ extract_references defined


In [7]:
from google.genai import types

CRAAP_PROMPT_V2 = """
You are a source verification system. Evaluate this reference using web search.

REFERENCE:
{reference_json}

Search for this source and evaluate on CRAAP:

**Currency (1-5):** How recent? Still relevant?
**Relevance (1-5):** Is this a substantive source for the topic?
**Authority (1-5):** Who wrote it? Are they credible?
**Accuracy (1-5):** Can claims be verified? Does it cite sources?
**Purpose (1-5):** Inform, persuade, or sell?

Also check: Can the URL be accessed?

Return JSON only:
{{
  "url_accessible": true/false/null,
  "currency": {{"score": 1-5, "evidence": "..."}},
  "relevance": {{"score": 1-5, "evidence": "..."}},
  "authority": {{"score": 1-5, "evidence": "..."}},
  "accuracy": {{"score": 1-5, "evidence": "..."}},
  "purpose": {{"score": 1-5, "evidence": "..."}},
  "overall_score": 1.0-5.0,
  "summary": "One sentence summary"
}}
"""

def verify_reference(reference: dict, model: str = "gemini-2.0-flash-exp") -> tuple[dict, LLMLog]:
    """Verify a single reference using search. Returns result and log."""
    
    prompt = CRAAP_PROMPT_V2.format(reference_json=json.dumps(reference, indent=2))
    
    start_time = time.time()
    
    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=types.GenerateContentConfig(
            tools=[types.Tool(google_search=types.GoogleSearch())]
        )
    )
    
    latency_ms = int((time.time() - start_time) * 1000)
    
    usage = response.usage_metadata
    log = LLMLog(
        timestamp=datetime.now().isoformat(),
        model=model,
        task='verify_reference',
        prompt_tokens=usage.prompt_token_count,
        completion_tokens=usage.candidates_token_count,
        total_tokens=usage.total_token_count,
        latency_ms=latency_ms
    )
    llm_logs.append(log)  # Now logging!
    
    try:
        clean_text = response.text.strip()
        if clean_text.startswith("```json"):
            clean_text = clean_text[7:]
        if clean_text.startswith("```"):
            clean_text = clean_text[3:]
        if clean_text.endswith("```"):
            clean_text = clean_text[:-3]
        
        return json.loads(clean_text.strip()), log
    except json.JSONDecodeError as e:
        print(f"Error: {e}\nRaw: {response.text}")
        return {}, log

print("✅ verify_reference defined (with logging)")

✅ verify_reference defined (with logging)


In [9]:
# Stage 1: Extract
print("=== STAGE 1: EXTRACTION ===")
refs, extract_log = extract_references(EXAMPLE_ANSWER)
print(f"Tokens: {extract_log.total_tokens} | Latency: {extract_log.latency_ms}ms")
print(f"Found {len(refs)} references\n")

# Stage 2: Verify each
print("=== STAGE 2: VERIFICATION ===")
verified = []
for i, ref in enumerate(refs):
    print(f"[{i+1}/{len(refs)}] {ref.get('platform_name', ref.get('url', 'unknown')[:30])}")
    result, log = verify_reference(ref)
    if result:
        print(f"       CRAAP: {result.get('overall_score', 'N/A')}/5 | {log.latency_ms}ms")
    verified.append({"reference": ref, "craap": result})

print("\n✅ Verification complete")

=== STAGE 1: EXTRACTION ===
Tokens: 5483 | Latency: 11000ms
Found 15 references

=== STAGE 2: VERIFICATION ===
[1/15] iTnews
       CRAAP: 4.4/5 | 5957ms
[2/15] Substack
       CRAAP: 4.0/5 | 4718ms
[3/15] The Conversation
       CRAAP: 4.6/5 | 4047ms
[4/15] Windows Forum
       CRAAP: 3.0/5 | 4997ms
[5/15] Deloitte
       CRAAP: 4.4/5 | 6557ms
[6/15] IT Brief Australia
       CRAAP: 4.4/5 | 6951ms
[7/15] PPC Land
       CRAAP: 4.4/5 | 6779ms
[8/15] The Missing Link
       CRAAP: 4.2/5 | 4922ms
[9/15] Digital
       CRAAP: 4.4/5 | 5889ms
[10/15] CFO Dive
       CRAAP: 4.6/5 | 6339ms
[11/15] Appinventiv
       CRAAP: 4.0/5 | 5825ms
[12/15] PitchBook
       CRAAP: 4.4/5 | 5419ms
[13/15] Mantel
       CRAAP: 4.6/5 | 5016ms
[14/15] Nicola Lazzari
       CRAAP: 4.4/5 | 5194ms
[15/15] Dasadvancedsystems
       CRAAP: 4.2/5 | 5191ms

✅ Verification complete


In [10]:
verified

[{'reference': {'raw_text': 'iTnews',
   'url': 'https://www.itnews.com.au/feature/how-australian-businesses-can-overcome-the-struggle-to-move-ai-from-hype-to-reality-618510',
   'doi': None,
   'arxiv_id': None,
   'source_type': 'news_media',
   'platform_name': 'iTnews',
   'stated_author': None,
   'stated_date': None,
   'stated_metrics': None},
  'craap': {'url_accessible': True,
   'currency': {'score': 5,
    'evidence': 'The article was published on 2025-07-09, making it recent and relevant in January 2026.'},
   'relevance': {'score': 5,
    'evidence': 'The article directly addresses the challenges Australian businesses face in implementing AI, which is a substantive topic.'},
   'authority': {'score': 4,
    'evidence': 'iTnews is a well-established Australian technology news publication. They have won awards for technology journalism and are considered a reliable source for IT professionals. Ry Crozier is the Editor in Chief. Velvet-Belle Templeman is the Publishing Direct

In [12]:
# let's try again with a stricter source checker.

CRAAP_PROMPT_STRICT = """
You are a SKEPTICAL source verification system. Your job is to be critical and identify weak sources.

REFERENCE:
{reference_json}

Search for this source and evaluate STRICTLY on CRAAP. Be harsh - most sources are mediocre.

SCORING GUIDE (err on the lower side):

**Currency (1-5):**
- 5: Published within last year, regularly updated
- 3: 1-3 years old, still somewhat relevant  
- 1: Outdated, stale, or no date available

**Relevance (1-5):**
- 5: Primary source, directly addresses topic with depth
- 3: Tangentially related, surface-level coverage
- 1: Barely relevant, clickbait title, or off-topic

**Authority (1-5):**
- 5: Named expert with verifiable credentials, institutional backing, peer-reviewed
- 3: Professional journalist at known outlet, or practitioner with some track record
- 2: Anonymous or pseudonymous author, no credentials stated
- 1: Random blog, no author info, self-published with no reputation

IMPORTANT: Personal blogs, Substack, Medium, etc. should START at 2 and only go higher if the author has VERIFIABLE expertise (real name, credentials, institutional affiliation). "Thought leaders" and "consultants" without specific credentials = 2.

**Accuracy (1-5):**
- 5: Cites primary sources, data is verifiable, peer-reviewed
- 3: Makes claims with some supporting links, but not rigorous
- 1: No citations, unverifiable claims, or contradicted by reliable sources

**Purpose (1-5):**
- 5: Educational, informational, no commercial motive
- 3: Some bias but primarily informative
- 1: Selling something, affiliate links, rage-bait, or propaganda

Also check: Is the URL accessible?

BE CRITICAL. A typical personal blog with no credentials should score 2-2.5 overall. Only authoritative, well-sourced content deserves 4+.

Return JSON only:
{{
  "url_accessible": true/false/null,
  "currency": {{"score": 1-5, "evidence": "..."}},
  "relevance": {{"score": 1-5, "evidence": "..."}},
  "authority": {{"score": 1-5, "evidence": "..."}},
  "accuracy": {{"score": 1-5, "evidence": "..."}},
  "purpose": {{"score": 1-5, "evidence": "..."}},
  "overall_score": 1.0-5.0,
  "red_flags": ["list", "any", "concerns"],
  "summary": "One critical sentence"
}}
"""

def verify_reference(reference: dict, model: str = "gemini-2.0-flash-exp") -> tuple[dict, LLMLog]:
    """Verify a single reference using search. Strict scoring."""
    
    prompt = CRAAP_PROMPT_STRICT.format(reference_json=json.dumps(reference, indent=2))
    
    start_time = time.time()
    
    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=types.GenerateContentConfig(
            tools=[types.Tool(google_search=types.GoogleSearch())]
        )
    )
    
    latency_ms = int((time.time() - start_time) * 1000)
    
    usage = response.usage_metadata
    log = LLMLog(
        timestamp=datetime.now().isoformat(),
        model=model,
        task='verify_reference',
        prompt_tokens=usage.prompt_token_count,
        completion_tokens=usage.candidates_token_count,
        total_tokens=usage.total_token_count,
        latency_ms=latency_ms
    )
    llm_logs.append(log)
    
    try:
        clean_text = response.text.strip()
        if clean_text.startswith("```json"):
            clean_text = clean_text[7:]
        if clean_text.startswith("```"):
            clean_text = clean_text[3:]
        if clean_text.endswith("```"):
            clean_text = clean_text[:-3]
        
        return json.loads(clean_text.strip()), log
    except json.JSONDecodeError as e:
        print(f"Error: {e}\nRaw: {response.text}")
        return {}, log

print("✅ verify_reference defined (strict mode)")

✅ verify_reference defined (strict mode)


In [13]:
# Stage 2: Verify try 2
print("=== STAGE 2: VERIFICATION ===")
verified = []
for i, ref in enumerate(refs):
    print(f"[{i+1}/{len(refs)}] {ref.get('platform_name', ref.get('url', 'unknown')[:30])}")
    result, log = verify_reference(ref)
    if result:
        print(f"       CRAAP: {result.get('overall_score', 'N/A')}/5 | {log.latency_ms}ms")
    verified.append({"reference": ref, "craap": result})

print("\n✅ Verification complete")

=== STAGE 2: VERIFICATION ===
[1/15] iTnews
       CRAAP: 3.8/5 | 5375ms
[2/15] Substack
       CRAAP: 3.0/5 | 5842ms
[3/15] The Conversation
       CRAAP: 3.8/5 | 4127ms
[4/15] Windows Forum
       CRAAP: 2.4/5 | 4573ms
[5/15] Deloitte
       CRAAP: 4.0/5 | 5161ms
[6/15] IT Brief Australia
       CRAAP: 3.8/5 | 5677ms
[7/15] PPC Land
       CRAAP: 3.8/5 | 4699ms
[8/15] The Missing Link
       CRAAP: 3.8/5 | 4445ms
[9/15] Digital
       CRAAP: 4.2/5 | 5466ms
[10/15] CFO Dive
       CRAAP: 3.8/5 | 4683ms
[11/15] Appinventiv
       CRAAP: 1.6/5 | 4935ms
[12/15] PitchBook


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota. Please migrate to Gemini 2.5 Flash Image (models/gemini-2.5-flash-image) for higher quota limits. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_requests_per_model', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash-exp'}, 'quotaValue': '10'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '0s'}]}}

In [14]:
llm_logs

[LLMLog(timestamp='2026-01-08T03:07:54.437699', model='gemini-2.0-flash-exp', task='extract_references', prompt_tokens=3667, completion_tokens=1816, total_tokens=5483, latency_ms=11000),
 LLMLog(timestamp='2026-01-08T03:08:00.395690', model='gemini-2.0-flash-exp', task='verify_reference', prompt_tokens=404, completion_tokens=376, total_tokens=780, latency_ms=5957),
 LLMLog(timestamp='2026-01-08T03:08:05.113982', model='gemini-2.0-flash-exp', task='verify_reference', prompt_tokens=380, completion_tokens=421, total_tokens=801, latency_ms=4718),
 LLMLog(timestamp='2026-01-08T03:08:09.161386', model='gemini-2.0-flash-exp', task='verify_reference', prompt_tokens=389, completion_tokens=331, total_tokens=720, latency_ms=4047),
 LLMLog(timestamp='2026-01-08T03:08:14.158766', model='gemini-2.0-flash-exp', task='verify_reference', prompt_tokens=389, completion_tokens=317, total_tokens=706, latency_ms=4997),
 LLMLog(timestamp='2026-01-08T03:08:20.716454', model='gemini-2.0-flash-exp', task='verif

In [15]:
# how to aggregate
def calculate_snifftest(verified_refs: list[dict]) -> dict:
    """Calculate Snifftest score from verified references."""
    
    scores = []
    red_flags = []
    
    for v in verified_refs:
        craap = v.get("craap", {})
        if craap and craap.get("overall_score"):
            scores.append(craap["overall_score"])
        if craap and craap.get("red_flags"):
            red_flags.extend(craap["red_flags"])
    
    if not scores:
        return {
            "score": 0,
            "label": "Unknown",
            "emoji": "❓",
            "scores": scores,
            "red_flags": red_flags
        }
    
    mean_score = sum(scores) / len(scores)
    min_score = min(scores)
    low_count = sum(1 for s in scores if s < 2.5)
    
    # Determine label (penalize if ANY source is bad)
    if min_score < 2 or low_count >= 2:
        label, emoji = "Foul", "🤢"
    elif mean_score < 2.5 or low_count >= 1:
        label, emoji = "Funky", "😬"
    elif mean_score < 3.5:
        label, emoji = "Fresh", "😊"
    else:
        label, emoji = "Sweet", "🌟"
    
    return {
        "score": round(mean_score, 2),
        "label": label,
        "emoji": emoji,
        "min_score": min_score,
        "num_refs": len(scores),
        "low_quality_count": low_count,
        "red_flags": list(set(red_flags))  # dedupe
    }

print("✅ calculate_snifftest defined")

✅ calculate_snifftest defined


In [16]:
SNIFFTEST_SUMMARY_PROMPT = """
You are the Snifftest summarizer. Given CRAAP evaluations of all sources in an AI-generated response, write a blunt, honest summary.

SNIFFTEST RESULT: {label} {emoji} ({score}/5)

VERIFIED SOURCES:
{verified_json}

Write a 2-3 sentence summary that:
1. States the overall verdict clearly
2. Calls out the weakest sources specifically
3. Notes any red flags
4. Suggests what would improve the response (if applicable)

Be direct and a little snarky. Examples:
- "This response smells funky. Half the sources are random blogs with no credentials..."
- "Pretty sweet! All sources are peer-reviewed or from reputable institutions..."
- "Foul. The main claim relies entirely on a 2019 Medium post by someone called 'CryptoGuru'..."

Keep it under 50 words. No JSON, just plain text.
"""

def generate_snifftest_summary(verified_refs: list[dict], snifftest: dict) -> tuple[str, LLMLog]:
    """Generate human-readable Snifftest summary."""
    
    prompt = SNIFFTEST_SUMMARY_PROMPT.format(
        label=snifftest["label"],
        emoji=snifftest["emoji"],
        score=snifftest["score"],
        verified_json=json.dumps(verified_refs, indent=2)
    )
    
    summary, log = generate_with_logging(prompt)
    return summary.strip(), log

print("✅ generate_snifftest_summary defined")

✅ generate_snifftest_summary defined


In [ ]:
# this code has not been executed, provided as a reference.
def run_snifftest(answer: str, verbose: bool = True) -> dict:
    """Run full Snifftest pipeline on an AI-generated answer."""
    
    results = {
        "input": answer,
        "references": [],
        "verified": [],
        "snifftest": {},
        "summary": "",
        "logs": []
    }
    
    # Stage 1: Extract
    if verbose:
        print("=== STAGE 1: EXTRACTION ===")
    refs, extract_log = extract_references(answer)
    results["references"] = refs
    results["logs"].append({"stage": "extract", **extract_log.to_dict()})
    
    if verbose:
        print(f"Found {len(refs)} references ({extract_log.total_tokens} tokens, {extract_log.latency_ms}ms)\n")
    
    if not refs:
        results["summary"] = "No references found to evaluate."
        return results
    
    # Stage 2: Verify each
    if verbose:
        print("=== STAGE 2: VERIFICATION ===")
    
    for i, ref in enumerate(refs):
        name = ref.get('platform_name') or ref.get('url', 'unknown')[:40]
        if verbose:
            print(f"[{i+1}/{len(refs)}] {name}...", end=" ")
        
        craap, verify_log = verify_reference(ref)
        results["verified"].append({"reference": ref, "craap": craap})
        results["logs"].append({"stage": f"verify_{i+1}", **verify_log.to_dict()})
        
        if verbose and craap:
            score = craap.get('overall_score', '?')
            print(f"CRAAP: {score}/5")
    
    # Stage 3: Calculate Snifftest
    if verbose:
        print("\n=== STAGE 3: SNIFFTEST ===")
    
    snifftest = calculate_snifftest(results["verified"])
    results["snifftest"] = snifftest
    
    if verbose:
        print(f"{snifftest['emoji']} {snifftest['label']}: {snifftest['score']}/5")
        print(f"   {snifftest['num_refs']} sources, {snifftest['low_quality_count']} low quality")
    
    # Stage 4: Generate summary
    if verbose:
        print("\n=== STAGE 4: SUMMARY ===")
    
    summary, summary_log = generate_snifftest_summary(results["verified"], snifftest)
    results["summary"] = summary
    results["logs"].append({"stage": "summary", **summary_log.to_dict()})
    
    if verbose:
        print(f"\n{snifftest['emoji']} SNIFFTEST: {snifftest['label'].upper()} ({snifftest['score']}/5)")
        print("-" * 50)
        print(summary)
    
    return results

print("✅ run_snifftest pipeline ready")

In [ ]:
results = run_snifftest(example_answer)

In [ ]:
print("=== DETAILED CRAAP SCORES ===\n")
for v in results["verified"]:
    ref = v["reference"]
    craap = v["craap"]
    
    name = ref.get("platform_name") or ref.get("url", "unknown")[:40]
    print(f"📎 {name}")
    print(f"   Type: {ref.get('source_type', 'unknown')}")
    
    if craap:
        print(f"   C:{craap.get('currency',{}).get('score','?')} "
              f"R:{craap.get('relevance',{}).get('score','?')} "
              f"A:{craap.get('authority',{}).get('score','?')} "
              f"A:{craap.get('accuracy',{}).get('score','?')} "
              f"P:{craap.get('purpose',{}).get('score','?')} "
              f"→ {craap.get('overall_score','?')}/5")
        if craap.get("red_flags"):
            print(f"   🚩 {', '.join(craap['red_flags'])}")
    print()

In [ ]:
total_tokens = sum(log.get("total_tokens", 0) for log in results["logs"])
total_latency = sum(log.get("latency_ms", 0) for log in results["logs"])

print(f"=== PIPELINE COST ===")
print(f"Total tokens: {total_tokens:,}")
print(f"Total latency: {total_latency:,}ms ({total_latency/1000:.1f}s)")
print(f"Calls: {len(results['logs'])}")